![title](img/graph.jpg)

In [2]:
import numpy as np
import enum

In [3]:
class State(enum.Enum):
    P000 = (0,0,0)
    P010 = (0,1,0)
    P001 = (0,0,1)
    P011 = (0,1,1)
    P111 = (1,1,1)
    P211 = (2,1,1)

In [4]:
p = 0.75
pi1 = 0.8
pi2 = 0.5

In [5]:
def event_happened(p):
    return np.random.uniform() > p 

def get_next_state(state):
    arrived = event_happened(p)
    channel1_done = event_happened(pi1)
    channel2_done = event_happened(pi2)
    if state == State.P000:
        if arrived:
            return State.P010
        else:
            return State.P000
    elif state == State.P010:
        if (not arrived and not channel1_done) or (arrived and channel1_done): #!!!!
            return State.P010
        if not arrived and channel1_done:
            return State.P000
        if arrived and not channel1_done:
            return State.P011      
    elif state == State.P001:
        if arrived and not channel2_done:
            return State.P011
        if arrived and channel2_done:
            return State.P010
        if not arrived and channel2_done:
            return State.P000
        if not arrived and not channel2_done:
            return State.P001
    elif state == State.P011:
        if not arrived and channel1_done and channel2_done:
            return State.P000
        if arrived and not channel1_done and not channel2_done:
            return State.P111
        if not arrived and channel1_done and not channel2_done:
            return State.P001
        if (not arrived and not channel1_done and channel2_done) \
            or (arrived and channel1_done and channel2_done):
            return State.P010
        if (not arrived and not channel1_done and not channel2_done) \
            or (arrived and channel1_done and not channel2_done) \
            or (arrived and not channel1_done and channel2_done): #!!!!
            return State.P011
    elif state == State.P111:
        if arrived and not channel1_done and not channel2_done:
            return State.P211
        if not arrived and channel1_done and channel2_done:
            return State.P010
        if (arrived and channel1_done and channel2_done) \
            or (not arrived and channel1_done and not channel2_done) \
            or (not arrived and not channel1_done and channel2_done):
            return State.P011
        if (not arrived and not channel1_done and not channel2_done) \
            or (arrived and channel1_done and not channel2_done) \
            or (arrived and not channel1_done and channel2_done): # !!!!
            return State.P111
    elif state == State.P211:
        if not arrived and channel1_done and channel2_done:
            return State.P011
        if (arrived and channel1_done and channel2_done) \
            or (not arrived and channel1_done and not channel2_done) \
            or (not arrived and not channel1_done and channel2_done):
            return State.P111
        if (arrived and not channel1_done and not channel2_done) \
            or (not arrived and not channel1_done and not channel2_done) \
            or (arrived and not channel1_done and channel2_done) \
            or (arrived and channel1_done and not channel2_done): # !!!!
            return State.P211
        

In [24]:
def get_out_order(prev, curr):
    prev = prev.value
    curr = curr.value
    first = (curr[1] - prev[1])
    first = first > 0 * first
    
    second = (curr[2] - prev[2])
    second = second > 0 * second
    
    return first + second + (curr[0] - prev[0]) 

def get_queue_amount(curr):
    curr = curr.value
    return curr[0]

def get_channel_load(curr, channel_num):
    curr = curr.value
    return curr[channel_num]
    

In [25]:
N = 100000

def model():
    L = 0
    A = 0
    K1 = 0
    K2 = 0
    P000 = P001 = P010 = P011 = P111 = P211 = 0
    state = State.P000

    for i in range(N):
        next_state = get_next_state(state)
        A += get_out_order(state, next_state)
        L += get_queue_amount(next_state)
        K1 += get_channel_load(next_state, 1)
        K2 += get_channel_load(next_state, 2)
        P000 += next_state == State.P000
        P001 += next_state == State.P001
        P010 += next_state == State.P010
        
        P011 += next_state == State.P011
        P111 += next_state == State.P111
        P211 += next_state == State.P211
        state = next_state
        
    A /= N
    L /= N
    K1 /= N
    K2 /= N
    P000 /= N
    P001 /= N
    P010 /= N
    P011 /= N
    P111 /= N
    P211 /= N
    
    print(P000+P001+P010+P011+P111+P211)
    print(f"A:{A} L:{L} K1:{K1} K2:{K2}")    
    print(f"P000:{P000} P001:{P001} P010:{P010} P011:{P011} P111:{P111} P211:{P211}")    

In [26]:
model()

1.0
A:0.17523 L:0.05131 K1:0.64157 K2:0.2453
P000:0.33721 P001:0.02122 P010:0.41749 P011:0.18041 P111:0.03603 P211:0.00764
